In [1]:
import os 
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json

In [2]:
o = pd.DataFrame()
for file in os.listdir('../ce_data/StatCache'):
    x = pd.read_csv("../ce_data/StatCache/"+file)
    x['id'] = x.index
    x['id'] = x['id'].apply(lambda x: file[:-4] + "_" + str(x))
    o = pd.concat([x, o], ignore_index=True)
o = o.sort_values("id", ascending=True, ignore_index=True).fillna(0)
print(o)
for i in ['32','64','128','256','512','1024','2048','4096','8192']:
    o['lru_'+i] = o['lru_'+i]/o['total_accesses']
print(o)

        random_32  lru_32  random_64  lru_64  random_128  lru_128  random_256  \
0        0.000000      16   0.000000       0    0.000000        0    0.000000   
1        0.000000      32   0.000000       0    0.000000        0    0.000000   
2        0.000000       0   0.000000       0    0.000000        0    0.000000   
3        0.000619     240   0.000497     240    0.000436      240    0.000436   
4        0.001229     416   0.000985     416    0.000924      416    0.000863   
...           ...     ...        ...     ...         ...      ...         ...   
141556   0.000000       0   0.000000       0    0.000000        0    0.000000   
141557   0.000000       0   0.000000       0    0.000000        0    0.000000   
141558   0.000000       0   0.000000       0    0.000000        0    0.000000   
141559   0.000000       0   0.000000       0    0.000000        0    0.000000   
141560   0.000000       0   0.000000       0    0.000000        0    0.000000   

        lru_256  random_512

In [3]:
print(o['lru_32'][0])

3.158958807177154e-05


In [4]:
cache_misses = pd.read_csv("../ce_data/caches/llc_miss_ratio.csv")
cache_misses = cache_misses.sort_values("id", ignore_index=True)
print(cache_misses)

                           id        32        64       128       256  \
0         400.perlbench-41B_0  1.000000  1.000000  1.000000  0.998652   
1         400.perlbench-41B_1  1.000000  1.000000  1.000000  1.000000   
2        400.perlbench-41B_10  1.000000  0.997375  0.997375  0.997375   
3       400.perlbench-41B_100  0.996656  0.996656  0.996656  0.993311   
4       400.perlbench-41B_101  1.000000  1.000000  1.000000  1.000000   
...                       ...       ...       ...       ...       ...   
141556        657.xz_s-56B_95  1.000000  1.000000  1.000000  1.000000   
141557        657.xz_s-56B_96  1.000000  1.000000  1.000000  1.000000   
141558        657.xz_s-56B_97  1.000000  1.000000  1.000000  1.000000   
141559        657.xz_s-56B_98  1.000000  1.000000  1.000000  1.000000   
141560        657.xz_s-56B_99  1.000000  1.000000  1.000000  1.000000   

             512      1024      2048      4096      8192  
0       0.889488  0.828841  0.803235  0.773585  0.768194  
1    

In [5]:
# Little check
d = {}
for i,j in zip(cache_misses["id"], o["id"]):
    if i != j:
        print(i)
    if i in d: 
        print(i)
    d[i]=1

In [6]:
accesses_l1d = pd.read_csv("../ce_data/caches/l1d_accesses.csv")
accesses_l1d = accesses_l1d.sort_values("id", ignore_index=True)
accesses_llc = pd.read_csv("../ce_data/caches/llc_accesses.csv")
accesses_llc = accesses_llc.sort_values("id", ignore_index=True)
print(accesses_llc)

                           id   32   64  128  256  512  1024  2048  4096  8192
0         400.perlbench-41B_0  742  742  742  742  742   742   742   742   742
1         400.perlbench-41B_1  911  911  911  911  911   911   911   911   911
2        400.perlbench-41B_10  381  381  381  381  381   381   381   381   381
3       400.perlbench-41B_100  299  299  299  299  299   299   299   299   299
4       400.perlbench-41B_101  370  370  370  370  370   370   370   370   370
...                       ...  ...  ...  ...  ...  ...   ...   ...   ...   ...
141556        657.xz_s-56B_95  838  838  838  838  838   838   838   838   838
141557        657.xz_s-56B_96  836  836  836  836  836   836   836   836   836
141558        657.xz_s-56B_97  836  836  836  836  836   836   836   836   836
141559        657.xz_s-56B_98  838  838  838  838  838   838   838   838   838
141560        657.xz_s-56B_99  836  836  836  836  836   836   836   836   836

[141561 rows x 10 columns]


In [11]:
from sklearn.metrics import mean_squared_error
for i in ['1024','2048','4096','8192']:
    tmp = o['lru_'+i] / o['lru_512']
    tmp = tmp.loc[accesses_l1d[i] < accesses_llc[i]]
    cms = cache_misses[accesses_l1d[i] < accesses_llc[i]]
    nans = tmp.isnull()
    c_acc_lru = mean_squared_error(tmp[~nans], cms[~nans][i])
    #c_acc_rand = mean_squared_error(o['random_'+i], cache_misses[i])
    print(f'Cache size {i}\t: lru_mse = {c_acc_lru}')

Cache size 1024	: lru_mse = 0.08082740451216865
Cache size 2048	: lru_mse = 0.07742140097492724
Cache size 4096	: lru_mse = 0.07624447396664075
Cache size 8192	: lru_mse = 0.08217087173089259


In [12]:
from sklearn.metrics import mean_squared_error
for i in ['1024','2048','4096','8192']:
    tmp = o['lru_'+i] * accesses_l1d[i]/accesses_llc[i]
    tmp = tmp.loc[accesses_l1d[i] < accesses_llc[i]]
    cms = cache_misses[accesses_l1d[i] < accesses_llc[i]]
    nans = tmp.isnull()
    c_acc_lru = mean_squared_error(tmp[~nans], cms[~nans][i])
    c_acc_rand = mean_squared_error(o['random_'+i], cache_misses[i])
    print(f'Cache size {i}\t: lru_mse = {c_acc_lru}')

Cache size 1024	: lru_mse = 0.02774767439905676
Cache size 2048	: lru_mse = 0.025546854956137056
Cache size 4096	: lru_mse = 0.026289475371176176
Cache size 8192	: lru_mse = 0.029374556693479742


In [9]:
df = pd.DataFrame()
for i in ['1024','2048','4096','8192']:
    df['caches_'+i] = cache_misses[i]
    print(accesses_l1d[accesses_l1d[i] < accesses_llc[i]])
    print(accesses_llc[accesses_l1d[i] < accesses_llc[i]])
    tmp = o['lru_'+i] * accesses_l1d[i]/accesses_llc[i]
    df['statcache_'+i] = tmp
    df['diff_'+i] = abs(df['caches_'+i] - df['statcache_'+i])

                          id      32      64     128     256     512    1024  \
4494           403.gcc-16B_0  118649  118665  118654  118677  118640  118793   
4495           403.gcc-16B_1  117266  117262  117271  117261  117263  117399   
4496          403.gcc-16B_10  123505  123526  123500  123507  123486  123733   
4498         403.gcc-16B_101  134420  134440  134423  134445  134446  134633   
4499         403.gcc-16B_102  118587  118593  118574  118551  118504  118660   
...                      ...     ...     ...     ...     ...     ...     ...   
134772   654.roms_s-1390B_73  120235  119373  119277  120201  119411  120139   
134802   654.roms_s-1390B_83   99945   99839  100118   99856   99964  100560   
134803   654.roms_s-1390B_84  117255  117573  117176  117397  117375  117599   
135035  654.roms_s-1613B_292  106619  106729  106889  106832  106923  106914   
137948   654.roms_s-842B_217  127041  127531  127036  127871  127199  126576   

          2048    4096    8192  
4494  